# Conversion de la base de données SQLite vers Delta Lake

**Objectif :** Ce notebook convertit les tables de la base de données SQLite `ELECTIONS.db` au format Delta Lake. Les tables Delta résultantes sont stockées localement, en préparation d'une utilisation sur des plateformes comme Databricks.

### Dépendances

La librairie `deltalake` est requise pour interagir avec le format Delta sans dépendre de Spark.

```bash
pip install deltalake
```

### Configuration

Définition des chemins d'accès pour la base de données SQLite source et le répertoire de sortie pour les tables Delta.

In [5]:
import sqlite3
import pandas as pd
from deltalake import write_deltalake
import os

# Chemin vers la base de données SQLite source.
SQLITE_DB_PATH = '../../database/ELECTIONS.db'

# Répertoire de destination pour les tables Delta.
DELTA_OUTPUT_DIR = '../../database/databricks/'

# Création du répertoire de sortie s'il n'existe pas.
os.makedirs(DELTA_OUTPUT_DIR, exist_ok=True)

print(f"Source: {SQLITE_DB_PATH}")
print(f"Destination: {DELTA_OUTPUT_DIR}")

Source: ../../database/ELECTIONS.db
Destination: ../../database/databricks/


### Script de conversion

Le script suivant exécute le processus de conversion :
1. Il se connecte à la base de données SQLite pour lister toutes les tables.
2. Pour chaque table, il charge les données dans un DataFrame Pandas.
3. Le DataFrame est ensuite écrit au format Delta Lake dans le répertoire de destination.

In [6]:
def convert_sqlite_to_delta(db_path, output_dir):
    """
    Convertit toutes les tables d'une base de données SQLite en tables Delta Lake.

    Args:
        db_path (str): Chemin vers le fichier de base de données SQLite.
        output_dir (str): Répertoire où stocker les tables Delta.
    """
    print("\n--- Démarrage de la conversion ---")
    
    try:
        # Connexion à la base de données SQLite pour récupérer les noms de tables.
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [row[0] for row in cursor.fetchall()]
        print(f"Tables à convertir : {tables}")
    except Exception as e:
        print(f"Erreur: Impossible de lire la liste des tables depuis la base de données : {e}")
        return

    # Itération sur chaque table pour la conversion.
    for table_name in tables:
        try:
            print(f"\nTraitement de la table '{table_name}'...")
            
            # Lecture de la table dans un DataFrame Pandas.
            df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
            
            # Définition du chemin de sortie pour la table Delta.
            table_path = os.path.join(output_dir, table_name)
            
            # Écriture du DataFrame au format Delta Lake.
            # Le mode 'overwrite' écrase la table si elle existe déjà,
            # permettant une ré-exécution idempotente du script.
            write_deltalake(table_path, df, mode='overwrite')
            
            print(f"✅ Table '{table_name}' convertie avec succès vers '{table_path}'")
            
        except Exception as e:
            print(f"❌ Erreur lors du traitement de la table '{table_name}': {e}")
            continue
            
    conn.close()
    print("\n--- Conversion terminée ! ---")

# Exécution de la fonction de conversion.
convert_sqlite_to_delta(SQLITE_DB_PATH, DELTA_OUTPUT_DIR)


--- Démarrage de la conversion ---
Tables à convertir : ['UNEMPLOYMENT_DATA', 'CRIME_DATA', 'ELECTION_DATA', 'IMMIGRATION_DATA', 'POVERTY_DATA', 'PROCESSED_TRAIN_DATA', 'PROCESSED_TEST_DATA', 'ELECTION_RESULTS_FOR_BI']

Traitement de la table 'UNEMPLOYMENT_DATA'...
✅ Table 'UNEMPLOYMENT_DATA' convertie avec succès vers '../../database/databricks/UNEMPLOYMENT_DATA'

Traitement de la table 'CRIME_DATA'...
✅ Table 'CRIME_DATA' convertie avec succès vers '../../database/databricks/CRIME_DATA'

Traitement de la table 'ELECTION_DATA'...
✅ Table 'ELECTION_DATA' convertie avec succès vers '../../database/databricks/ELECTION_DATA'

Traitement de la table 'IMMIGRATION_DATA'...
✅ Table 'IMMIGRATION_DATA' convertie avec succès vers '../../database/databricks/IMMIGRATION_DATA'

Traitement de la table 'POVERTY_DATA'...
✅ Table 'POVERTY_DATA' convertie avec succès vers '../../database/databricks/POVERTY_DATA'

Traitement de la table 'PROCESSED_TRAIN_DATA'...
✅ Table 'PROCESSED_TRAIN_DATA' convertie 

### Vérification

Après l'exécution du script, le répertoire de sortie (`../../database/databricks/`) contiendra un sous-dossier pour chaque table convertie. Chaque sous-dossier représente une table Delta et inclut des fichiers de données Parquet ainsi qu'un répertoire `_delta_log` contenant le journal des transactions.